## **Activity 17.01**
### Building a Classification Model with dataset given

### Importing modules

In [4]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np


### Installing Featuretools

In [ ]:
!pip3 install featuretools

### Importing featuretools

In [6]:
import featuretools as ft

/usr/local/lib/python3.7/dist-packages/woodwork/__init__.py:23: FutureWarning: Woodwork may not support Python 3.7 in next non-bugfix release.
  "Woodwork may not support Python 3.7 in next non-bugfix release.", FutureWarning
/usr/local/lib/python3.7/dist-packages/featuretools/__init__.py:67: FutureWarning: Featuretools may not support Python 3.7 in next non-bugfix release.
  FutureWarning,


### Read in the data

In [7]:

clients = pd.read_csv('https://raw.githubusercontent.com/fenago/DSBook/main/Chapter%2017/clients.csv', parse_dates = ['joined'])
loans = pd.read_csv('https://raw.githubusercontent.com/fenago/DSBook/main/Chapter%2017/loans.csv', parse_dates = ['loan_start', 'loan_end'])
payments = pd.read_csv('https://raw.githubusercontent.com/fenago/DSBook/main/Chapter%2017/payments.csv', parse_dates = ['payment_date'])

In [ ]:
clients.head()

In [ ]:
loans.sample(10)

In [ ]:
payments.sample(10)

### Creating  a month column and a log of income column

In [ ]:

clients['join_month'] = clients['joined'].dt.month

clients['log_income'] = np.log(clients['income'])

clients.head()

### Groupby client id and calculate mean, max, min previous loan size

In [ ]:

stats = loans.groupby('client_id')['loan_amount'].agg(['mean', 'max', 'min'])
stats.columns = ['mean_loan_amount', 'max_loan_amount', 'min_loan_amount']
stats.head()

### Merge with the clients dataframe

In [ ]:

clients.merge(stats, left_on = 'client_id', right_index=True, how = 'left').head(10)

In [14]:
es = ft.EntitySet(id = 'clients')

### Create an entity from the client dataframe. This dataframe already has an index and a time index

In [18]:

es = es.add_dataframe(dataframe_name = 'clients', dataframe = clients, index = 'client_id', time_index = 'joined')

### Create an entity from the loans dataframe. This dataframe already has an index and a time index

In [22]:

es = es.add_dataframe(dataframe_name = 'loans', dataframe = loans, \
                              index = 'loan_id', \
                              time_index = 'loan_start')

###  Create an entity from the payments dataframe. This does not yet have a unique index

In [23]:

es = es.add_dataframe(dataframe_name = 'payments', 
                              dataframe = payments,
                              make_index = True,
                              index = 'payment_id',
                              time_index = 'payment_date')

In [ ]:
es

In [ ]:
es['loans']

### Relationship between clients and previous loans

In [28]:

# Add the relationship to the entity set
es = es.add_relationship('clients', 'client_id', 'loans', 'client_id' )

### Relationship between previous loans and previous payments

In [29]:
# Add the relationship to the entity set
es = es.add_relationship('loans', 'loan_id', 'payments', 'loan_id' )

es

Entityset: clients
  DataFrames:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id
    payments.loan_id -> loans.loan_id